# Roche robotics poject

## Imports

In [31]:
import random

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import torch
import torch.nn.functional as F
import json
import pandas as pd
import os

from habitat_baselines.config.default import get_config
from habitat_baselines.rl.ppo.ppo_trainer import PPOTrainer
from habitat_baselines.common.construct_vector_env import construct_envs
from habitat_baselines.config.default import get_config

from pg.base_pg import BasePolicyGradient
from pg.base_pg_trainer import BasePolicyGradientTrainer

from tensorboard.backend.event_processing import event_accumulator

Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail


## Useful functions

In [32]:
def parse_tensorboard(path, scalars, prefix=""):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _ = ea.Reload()
    df = pd.DataFrame(ea.Scalars(scalars[0]))
    df[f'{scalars[0]}'] = df['value']
    df.drop(columns=['value'], inplace=True)
    for k in scalars:
        df[k] = pd.DataFrame(ea.Scalars(k))['value']
    return df

## Configuration 1 : PointNav, Empty Room

### PPO Configuration

In [35]:


def build_PPO_config():    
    config = get_config("targetnav/comm_proj_discrete.yaml")
    #pointnav/ppo_pointnav.yaml
    #targetnav/comm_proj_discrete.yaml
    OmegaConf.set_readonly(config, False)
    config.habitat_baselines.tensorboard_dir = "tb/PPO"
    config.habitat_baselines.checkpoint_folder = "data/PPO4_checkpoints"
    config.habitat_baselines.num_updates = -1
    config.habitat_baselines.num_environments = 2
    config.habitat_baselines.verbose = True
    config.habitat_baselines.num_checkpoints = -1
    config.habitat_baselines.checkpoint_interval = 1000000
    config.habitat_baselines.total_num_steps = 150 * 1000
    config.habitat.dataset.data_path = "data/datasets/pointnav/simple_room/v0/{split}/empty_room.json.gz"
    config.habitat_baselines.load_resume_state_config = False
    #config.habitat.simulator
    
    #config.habitat.simulator.scene = "data/scene_datasets/simple_room/empty_room.glb"
    #config.habitat.simulator.scene = "data/scene_datasets/simple_room/empty_room.scene_instance.json"

    OmegaConf.set_readonly(config, True)

    return config

config = build_PPO_config()
om_config = OmegaConf.to_yaml(config)
#print(om_config)
      
# Set randomness
random.seed(config.habitat.seed)
np.random.seed(config.habitat.seed)
torch.manual_seed(config.habitat.seed)
if (
    config.habitat_baselines.force_torch_single_threaded
    and torch.cuda.is_available()
):
    torch.set_num_threads(1)

os.environ["MAGNUM_LOG"] = "quiet"
os.environ["HABITAT_SIM_LOG"] = "quiet"

In [18]:
import habitat_sim

backend_cfg = habitat_sim.SimulatorConfiguration()
print(backend_cfg)

sem_cfg = habitat_sim.CameraSensorSpec()
sem_cfg.uuid = "semantic"
sem_cfg.sensor_type = habitat_sim.SensorType.SEMANTIC
print(sem_cfg)

agent_cfg = habitat_sim.agent.AgentConfiguration()
agent_cfg.sensor_specifications = [sem_cfg]
print(agent_cfg)

AgentConfiguration(height=1.5, radius=0.1, sensor_specifications=[<habitat_sim._ext.habitat_sim_bindings.CameraSensorSpec object at 0x7f8f027ddc20>], action_space={'move_forward': ActionSpec(name='move_forward', actuation=ActuationSpec(amount=0.25, constraint=None)), 'turn_left': ActionSpec(name='turn_left', actuation=ActuationSpec(amount=10.0, constraint=None)), 'turn_right': ActionSpec(name='turn_right', actuation=ActuationSpec(amount=10.0, constraint=None))}, body_type='cylinder')


### Training phase

In [36]:
# Build the trainer and start training
trainer = PPOTrainer(config)
trainer.train()

#trainer.agent.ini

2025-05-16 09:52:25,698 config: habitat_baselines:
  trainer_name: ppo
  torch_gpu_id: 0
  video_render_views: []
  tensorboard_dir: tb/PPO
  writer_type: wb
  video_dir: video_dir
  video_fps: 10
  test_episode_count: -1
  eval_ckpt_path_dir: data/new_checkpoints
  num_environments: 2
  num_processes: -1
  checkpoint_folder: data/PPO4_checkpoints
  num_updates: -1
  num_checkpoints: -1
  checkpoint_interval: 1000000
  total_num_steps: 150000.0
  log_interval: 25
  video_interval: 150
  log_file: train.log
  force_blind_policy: false
  verbose: true
  eval_keys_to_include_in_name: []
  force_torch_single_threaded: true
  wb:
    project_name: ''
    entity: ''
    group: ''
    run_name: ''
    run_id: ''
  load_resume_state_config: false
  eval:
    split: val
    use_ckpt_config: true
    should_load_ckpt: true
    evals_per_ep: 1
    video_option: []
  profiling:
    capture_start_step: -1
    num_steps_to_capture: -1
  launch_eval_afterwards: true
  rl:
    preemption:
      append

ValueError: Argument 'target_id' must be set

In [12]:
type(trainer.envs)

NoneType

In [ ]:
sim_rom = RunTimeObjectManager(sim)
    sim_rom.delete_added_target()
    sim_rom.add_target('/scratch/singh/morph_ws/habitat-lab/data/objects/targets/red_ball.glb', [0.0, 0.0, 0.0])
    # '/workspace/habitat-sim/data/test_assets/objects/red_ball.glb'
    sim.step_physics(physics_step_time)

In [ ]:
ppo_config_1_logs = parse_tensorboard("tb/PPO", ["metrics/spl", "reward", "learner/grad_norm"])

### Agent sensors

In [ ]:
config = build_PPO_config()
OmegaConf.set_readonly(config, False)

# Creating the environment
envs = construct_envs(config)

# Retrieve the observations
obs = envs.reset()  

# Verify the keys of the observation
print("Available observations :", obs[0].keys())  # Obs of environment 0
OmegaConf.set_readonly(config, True)


## Configuration 2 : PointNav, Multiple Rooms

### PPO Configuration

In [ ]:
def build_PPO_config():
    config = get_config("pointnav/ppo_pointnav.yaml")
    OmegaConf.set_readonly(config, False)
    config.habitat_baselines.tensorboard_dir = "tb/PPO"
    config.habitat_baselines.num_updates = -1
    config.habitat_baselines.num_environments = 2
    config.habitat_baselines.verbose = True
    config.habitat_baselines.num_checkpoints = -1
    config.habitat_baselines.checkpoint_interval = 1000000
    config.habitat_baselines.total_num_steps = 150 * 1000
    config.habitat.dataset.data_path="data/datasets/pointnav/multiple_room/v0/{split}/Adrian.json.gz"
    OmegaConf.set_readonly(config, True)

    return config

config = build_PPO_config()  # Build the config for PPO
# Set randomness
random.seed(config.habitat.seed)
np.random.seed(config.habitat.seed)
torch.manual_seed(config.habitat.seed)
if (
    config.habitat_baselines.force_torch_single_threaded
    and torch.cuda.is_available()
):
    torch.set_num_threads(1)

os.environ["MAGNUM_LOG"] = "quiet"
os.environ["HABITAT_SIM_LOG"] = "quiet"

### Training phase

In [ ]:
# Build the trainer and start training
trainer = PPOTrainer(config)
trainer.train()

In [ ]:
ppo_config_2_logs = parse_tensorboard("tb/PPO", ["metrics/spl", "reward", "learner/grad_norm"])

## Perfomances

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6, 6))
plt.subplots_adjust(hspace=0.5)
plt.axes(axes[0])

plt.plot(ppo_config_1_logs['metrics/spl'], label="PPO")
plt.plot(ppo_config_2_logs['metrics/spl'], label="PPO")

plt.title("SPL")
plt.xlabel("Steps")
plt.legend()

plt.axes(axes[1])

plt.plot(ppo_config_1_logs['learner/grad_norm'], label="PPO")
plt.plot(ppo_config_2_logs['learner/grad_norm'], label="PPO")

plt.title("Gradient Norm")
plt.xlabel("Steps")
plt.legend()